In [1]:
using MNIST

In [2]:
imageTrain,labelTrain  = traindata();  
imageTest, labelTest= testdata();  


In [3]:
trainSize = size(imageTrain); 

In [4]:
trainSize

(784,60000)

In [5]:
testSize = size(imageTest);  

In [6]:
testSize

(784,10000)

In [7]:
 
if trainSize[2] != length(labelTrain)
    println("Train image and train label size mismatches!");
end
if testSize[2] != length(labelTest)
    println("Test image and Test label size mismatches!");
end
if trainSize[1] != testSize[1]
    println("Train and test image size mismatches!");
end

In [8]:
N = trainSize[2];  
 TOTAL_HU = 300;  
TOTAL_IN = trainSize[1];  
 TOTAL_OUT = 10;  
 MAX_ITERATION = 20;  
 BETA = 0.01; # Scaling factor in sigmoid function  
 ALPHA = 0.1; # Learning rate  

0.1

In [9]:
# Initialize the weights from Uniform(0, 1)  
 w1 = rand(TOTAL_IN + 1, TOTAL_HU); # Weight for layer-1, including bias unit  

785x300 Array{Float64,2}:
 0.629901   0.22127    0.447816   …  0.0487561  0.321322   0.846342 
 0.710614   0.072906   0.591607      0.995368   0.186928   0.161391 
 0.49941    0.986222   0.305839      0.224042   0.202784   0.565619 
 0.968363   0.510067   0.221911      0.279915   0.0572508  0.864926 
 0.364337   0.0738831  0.524555      0.498164   0.514513   0.144201 
 0.432102   0.248347   0.0823544  …  0.60979    0.982351   0.529406 
 0.370277   0.40129    0.0290316     0.542595   0.655963   0.960837 
 0.708008   0.0658885  0.373793      0.0543853  0.983662   0.782972 
 0.511369   0.754801   0.660955      0.649942   0.784718   0.661414 
 0.863765   0.73841    0.368305      0.259201   0.899278   0.871664 
 0.979994   0.293976   0.621154   …  0.500252   0.383564   0.228324 
 0.0433023  0.54015    0.395065      0.643655   0.607078   0.321662 
 0.259014   0.665909   0.33658       0.96577    0.391626   0.51544  
 ⋮                                ⋱                                 
 0.90491

In [10]:
w2 = rand(TOTAL_HU + 1, TOTAL_OUT); # Weight for layer-2. including bias unit

301x10 Array{Float64,2}:
 0.117041    0.934401   0.929163    …  0.66628    0.0104957  0.350227 
 0.0587407   0.946799   0.535678       0.286989   0.644778   0.832318 
 0.241341    0.83246    0.324461       0.54547    0.604117   0.979045 
 0.688548    0.405992   0.0926827      0.977081   0.669024   0.999684 
 0.730682    0.0791037  0.022243       0.642939   0.0943873  0.710063 
 0.853885    0.0622337  0.254502    …  0.834986   0.759008   0.456849 
 0.905157    0.640423   0.259547       0.954051   0.0896665  0.656295 
 0.744066    0.701024   0.863848       0.108702   0.633445   0.922629 
 0.167682    0.342231   0.289882       0.337194   0.605648   0.814326 
 0.869086    0.58785    0.896883       0.587486   0.473591   0.122626 
 0.827224    0.0494885  0.500679    …  0.509234   0.570346   0.199959 
 0.393255    0.307885   0.186024       0.455651   0.244308   0.336979 
 0.0578274   0.753335   0.183693       0.589652   0.638697   0.916624 
 ⋮                                  ⋱               

In [11]:
# Temporary matrix to store weight updates and activations  
 x1 = zeros(TOTAL_IN + 1, 1); 

In [12]:
 x2 = zeros(TOTAL_HU + 1, 1); 

In [13]:
x22 = zeros(TOTAL_HU, 1);

In [14]:
 x3 = zeros(TOTAL_OUT, 1); 

In [15]:
e1 = zeros(TOTAL_IN + 1, 1);

In [16]:
e2 = zeros(TOTAL_HU + 1, 1); 

In [17]:
e3 = zeros(TOTAL_OUT, 1);

In [18]:
dw1 = zeros(TOTAL_IN + 1, TOTAL_HU);

In [21]:
dw2 = zeros(TOTAL_HU + 1, TOTAL_OUT);

In [20]:
function sigmoidGradient(z)
  return sigmoid(z).*(1-sigmoid(z));
end

sigmoidGradient (generic function with 1 method)

In [22]:
function sigmoid(z)
    g = 1.0 ./ (1.0 + exp(-z));
    return g;
end

sigmoid (generic function with 1 method)

In [23]:
 A = eye(TOTAL_OUT);  
 success = zeros(MAX_ITERATION, 1);  
 # Training  
 for t = 1:MAX_ITERATION  
    println("Iteration: ", t);  
   perm = randperm(N);  
   for p = 1:N  
     if (mod(p, 1000) == 0)  
            println(p);  
     end  
     # Propogate forward  
     index = perm(p);  
     x1 = [imageTrain(:, index); 1];  
     x22 = sigmoid(w1' * x1);  
     x2 = [x22; 1];  
     x3 = sigmoid(w2' * x2);  
     # Propogate backward  
     e3 = x3 - A(:, labelTrain(index) + 1);  
     e2 = w2 * (e3 .* x3 .* (1 - x3));  
     e2 = e2(1:TOTAL_HU);  
     e1 = w1 * e2;  
     # Update weight  
     w2 = w2 - ALPHA * x2 * (e3 .* x3 .* (1 - x3))';  
     w1 = w1 - ALPHA * x1 * (e2 .* x22 .* (1 - x22))';  
   end  
   # Check training error  
   success(t) = 0;  
   for i = 1:N  
     x1 = [imageTrain(:, i); 1];  
     x2 = [sigmoid(w1' * x1); 1];  
     x3 = sigmoid(w2' * x2);  
     [dummy, m] = max(x3);  
     if (m == labelTrain(i) + 1)  
       success(t) = success(t) + 1;  
     end  
   end  
 end  
 testSuccess = 0;  
 for i = 1:testSize(2)  
   x1 = [imageTest(:, i); 1];  
   x2 = [sigmoid(w1' * x1); 1];  
   x3 = sigmoid(w2' * x2);  
   [dummy, m] = max(x3);  
   if (m == labelTest(i) + 1)  
     testSuccess = testSuccess + 1;  
   end  
 end  

LoadError: syntax: use "(a, b) = ..." to assign multiple values
while loading In[23], in expression starting on line 38

In [2]:
AA = [[1,2] [3,4]]

2x2 Array{Int64,2}:
 1  3
 2  4

In [5]:
BB = 0

0

In [6]:
BB = AA

2x2 Array{Int64,2}:
 1  3
 2  4

In [8]:
AA*BB

2x2 Array{Int64,2}:
  8  12
 12  18

In [13]:
AA.*2

2x2 Array{Int64,2}:
 2  6
 4  8

In [15]:
sum(sum(AA))

10

In [16]:
maximum(AA)

4

In [ ]:
# learning rate
alpha = 0.1;
# number of iterations
epoch = 10;
# cost per epoch
J = zeros(epoch,1);
# ====================================================================
# Train the neural network using feedforward-backpropagation algorithm
# ====================================================================
for i = 1:epoch
    # ===================
    # Feedforward process
    # ===================
    # input layer
    # add one bias element
    x1 = [ones(1,size(X,2)), X]; #(785x60000)

    # hidden layer
    s2 = Theta1'*x1; #(300x60000)
    x2 = sigmoid(s2); 
    # add one bias element
    x2 = [ones(1,size(x2,2)), x2]; #(26x60000)

    # output layer
    s3 = Theta2'*x2; #(10x60000)
    x3 = sigmoid(s3);
    
    # ===========================
    # find cost of the new output
    # ===========================
    J[i,:] = costFunction(Y, x3);
    
    # =======================
    # Backpropagation process
    # =======================
    inputLayerRegularizedGradient = zeros(size(Theta1));
    hiddenLayerRegularizedGradient = zeros(size(Theta2));
        
    # delta for output layer
    delta3 = x3 - Y; #(10x60000)
    # gradient for hidden layer
    reg_theta2 = ((lambda/m)*Theta2);
    reg_theta2[:,1] = 0;
    hiddenLayerRegularizedGradient = x2*delta3'; #(26x10)
    # delta for hidden layer
    delta2 = (Theta2[2:end,:]*delta3).*sigmoidGradient(s2) ; #(25x60000)
    # gradient for input layer
    reg_theta1 = ((lambda/m)*Theta1);
    reg_theta1[1,:] = 0;
    inputLayerRegularizedGradient = x1*delta2'; #(785x25)
    # there is no delta term for the input layer
        
    # adjust the weights (thetas)
    Theta1 = Theta1 - alpha* inputLayerRegularizedGradient*(1/m);
    Theta2 = Theta2 - alpha* hiddenLayerRegularizedGradient*(1/m);  
end

In [1]:
AA

LoadError: AA not defined
while loading In[1], in expression starting on line 1